In [ ]:
#changing image orientation
src = cv.imread{read_image}
def drawAxis(img, p_, q_, colour, scale):
    p = list(p_)
    q = list(q_)
    
    angle = atan2(p[1] - q[1], p[0] - q[0]) # angle in radians
    hypotenuse = sqrt((p[1] - q[1]) * (p[1] - q[1]) + (p[0] - q[0]) * (p[0] - q[0]))
    # Here we lengthen the arrow by a factor of scale
    q[0] = p[0] - scale * hypotenuse * cos(angle)
    q[1] = p[1] - scale * hypotenuse * sin(angle)
    cv.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), colour, 1, cv.LINE_AA)
    # create the arrow hooks
    p[0] = q[0] + 9 * cos(angle + pi / 4)
    p[1] = q[1] + 9 * sin(angle + pi / 4)
    cv.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), colour, 1, cv.LINE_AA)
    p[0] = q[0] + 9 * cos(angle - pi / 4)
    p[1] = q[1] + 9 * sin(angle - pi / 4)
    cv.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), colour, 1, cv.LINE_AA)

In [ ]:
def getOrientation(pts, img):    
    sz = len(pts)
    data_pts = np.empty((sz, 2), dtype=np.float64)
    for i in range(data_pts.shape[0]):
        data_pts[i,0] = pts[i,0,0]
        data_pts[i,1] = pts[i,0,1]
    # Perform PCA analysis
    mean = np.empty((0))
    mean, eigenvectors, eigenvalues = cv.PCACompute2(data_pts, mean)

    # Store the center of the object
    cntr = (int(mean[0,0]), int(mean[0,1]))

    cv.circle(img, cntr, 3, (255, 0, 255), 2) #Draw a circle at the center of PCA
    p1 = (cntr[0] + 0.02 * eigenvectors[0,0] * eigenvalues[0,0], cntr[1] + 0.02 * eigenvectors[0,1] * eigenvalues[0,0])
    p2 = (cntr[0] - 0.02 * eigenvectors[1,0] * eigenvalues[1,0], cntr[1] - 0.02 * eigenvectors[1,1] * eigenvalues[1,0])
    drawAxis(img, cntr, p1, (0, 255, 0), 1) # , 
    drawAxis(img, cntr, p2, (255, 255, 0), 1) #yellow
    angle = atan2(eigenvectors[0,1], eigenvectors[0,0]) # orientation in radians #PCA first dimension angle    
    return angle

In [ ]:
def rotate_img(img,angle):
    """ center rotates the image, the input angle is radians """
    angle_o=(angle-pi/2)*180/pi # 
    height = img.shape[0]  # original image height
    width = img.shape[1]  # original image width
    rotateMat = cv.getRotationMatrix2D((width / 2, height / 2), angle_o, 1)  # Rotate the image by angle
    heightNew = int(width * math.fabs(math.sin(angle)) + height * math.fabs(math.cos(angle)))
    widthNew = int(height * math.fabs(math.sin(angle)) + width * math.fabs(math.cos(angle)))

    rotateMat[0, 2] += (widthNew - width) / 2  
    rotateMat[1, 2] += (heightNew - height) / 2
    imgRotation = cv.warpAffine(img, rotateMat, (widthNew, heightNew), borderValue=(255, 255, 255))
    plt.figure(4)
    plt.imshow(imgRotation,cmap='gray')
    return imgRotation
parser = argparse.ArgumentParser()
args, unknown = parser.parse_known_args()

if src is None:
    print('Could not open or find the image: ', args.input)
    exit(0)

plt.figure(1)

# Convert image to grayscale
gray = cv.cvtColor(src, cv.COLOR_BGR2GRAY)

# Convert image to binary
_, bw = cv.threshold(gray, 150, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)
plt.figure(2);plt.imshow(bw,cmap='gray')
contours, _ = cv.findContours(bw, cv.RETR_LIST, cv.CHAIN_APPROX_NONE)
areas=[]
areas_angle=[]
for i, c in enumerate(contours):
    # Calculate the area of each contour
    area = cv.contourArea(c)
    # Ignore contours that are too small or too large
    if area < 1e2 or 1e5 < area:
        continue
    areas.append(area)
    # Draw each contour only for visualisation purposes
    cv.drawContours(src, contours, i, (0, 0, 255), 2)
    # Find the orientation of each shape
    angle=getOrientation(c, src)
    areas_angle.append(angle)

plt.figure(3)

plt.imshow(src,cmap='gray')


# Calculate the direction of the largest connected domain
ind=np.argmax(areas)
imgRotation=rotate_img(src,areas_angle[ind])
plt.show()
print("Task done")
    